First create the symbols

In [1]:
# Create symbols

Then create modified DH parameters

In [2]:
# Create Modified DH parameters

Define the transformation matrix

In [3]:
# Define Modified DH Transformation matrix

Create individual transformation matrices

In [4]:
# Create individual transformation matrices

Extract End-Effector position and orient from request

In [6]:
# Extract end-effector position and orientation from request

Calculate joint angles using Geometric IK method

In [7]:
# Calculate joint angles using Geometric IK method